Load data 

In [1]:
import midaa as maa
import scanpy as sc

# Load the real dataset
adata_beta_hfd  = sc.read_h5ad(r"pbmc3k_processed.h5ad")

# Convert to MIDAA input
input_matrix, norm_factors, input_distribution = maa.get_input_params_adata(adata_beta_hfd)

print(len(input_matrix))         # should be 1
print(input_matrix[0].shape)     # should be (2638, 1838)

# Number of archetypes
narchetypes = 3


print(len(norm_factors))
print(norm_factors[0].shape)
print(norm_factors[0][:5])  # first 5 values
print(input_distribution)
# For RNA: usually ['NB']


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'pbmc3k_processed.h5ad', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

Run model

In [ ]:


# Number of archetypes
narchetypes = 3

# Run MIDAA
aa_result = maa.fit_MIDAA(
    input_matrix,
    norm_factors,
    input_distribution,
    narchetypes=narchetypes
)




ELBO: 29600288.00000  : 100%|██████████| 2000/2000 [22:30<00:00,  1.48it/s]
c:\Users\Manuel\anaconda3_new\envs\MIDAA\Lib\site-packages\pyro\primitives.py:163: RuntimeWarning: trying to observe a value outside of inference at loss
  warnings.warn(


AttributeError: 'list' object has no attribute 'shape'

Look at code results

In [1]:
import numpy as np

def extract_mida_matrices(aa_result, input_matrix):
    iq = aa_result["inferred_quantities"]

    # A: (n_cells × n_archetypes)
    A = iq["A"]

    # Z: latent representation (n_cells × latent_dim)
    Z = iq["Z"]

    # B: archetype positions in latent space (n_archetypes × latent_dim)
    B = iq["B"]

    # archetypes_inferred: gene weights for each archetype
    # shape: (n_genes, n_archetypes)
    C = iq["archetypes_inferred"]

    # X: original input data
    X = input_matrix[0]   # (n_cells × n_genes)

    # Labels = archetype with highest membership
    labels = np.argmax(A, axis=1)

    return A, B, Z, C, X, labels

A, B, Z, C, X, labels = extract_mida_matrices(aa_result, input_matrix)

print("A (memberships):", A.shape)
print("B (latent archetype coords):", B.shape)
print("Z (latent cells):", Z.shape)
print("C (gene weights):", C.shape)
print("X (input data):", X.shape)
print("labels:", labels.shape)

import torch

# Save   three core MIDAA matrices
torch.save(
    {'A': A, 'B': B, 'C': C},
    "midaa_core_matrices.pth"
)

print("Saved A, B, C matrices to midaa_core_matrices.pth")


import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=(7, 6))
plt.scatter(
    Z[:, 0], Z[:, 1],
    c=labels,
    s=10,
    cmap="viridis",
    alpha=0.7
)

plt.title("MIDAA Latent Space (Z) — Cells Colored by Archetype")
plt.xlabel("Z1")
plt.ylabel("Z2")
plt.colorbar(label="Archetype")
plt.tight_layout()
plt.show()


NameError: name 'aa_result' is not defined